In [2]:
import sqlite3
from urllib.parse import urljoin, urlparse
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [3]:
conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

In [4]:
cur.executescript('''
CREATE TABLE IF NOT EXISTS Pages (
                  id INTEGER PRIMARY KEY,
                  url TEXT UNIQUE,
                  html TEXT,
                  error INTEGER, old_rank REAL, new_rank REAL
                  );
CREATE TABLE IF NOT EXISTS Links (
                  from_id INTEGER, to_id INTEGER, UNIQUE(from_id,to_id)
                  );
CREATE TABLE IF NOT EXISTS Webs (url TEXT UNIQUE);''')

In [5]:
cur.execute('SELECT id, url FROM Pages WHERE html IS NOT NULL AND error IS NULL ORDER BY RANDOM() LIMIT 1')
row = cur.fetchone()

if row is not None:
    print('Restarting existing crwal, Remove spider.sqlite to start a fresh crawl.')
else:
    starturl = input('Enter web url or press enter')
    if len(starturl) < 1: starturl = 'http://www.dr-chuck.com/'
    if starturl.endswith('/'): starturl = starturl[:-1]
    web = starturl

    if (starturl.endswith('html' or starturl.endswith('htm'))) :
        pos = starturl.rfind('/')
        web = starturl[:pos]

    if len(web) > 1:
        cur.execute('INSERT OR IGNORE INTO Webs VALUES (?)',(web,))
        cur.execute('INSERT OR IGNORE INTO Pages (url,html,new_rank) VALUES (?, NULL, 1.0)',(we))